# Sentinel-2 L2A Annual Composite

In [ ]:
from dotenv import load_dotenv

load_dotenv()

import os

import fsspec
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import numpy as np
import odc.stac
import pystac
import rioxarray
import shapely
import stac_geoparquet
import xarray as xr
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

## Read a snapshot with the spatial extents of all tiles

Connect to the CoCliCo STAC and read the spatial extents using stac-geoparquet.

In [ ]:
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)

collection = coclico_catalog.get_child("s2-l2a-composite")

snapshot = read_snapshot(collection, columns=None, storage_options=None)

snapshot.head()

In [ ]:
import panel as pn
pn.Column(snapshot[["geometry"]].hvplot(geo=True, fill_color=None, width=500, tiles="OSM")).show()

## Zoom to your area of interest

In [ ]:
m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = m.center = (53.19, 4.9)
m.zoom = 12
m.layout.height = "800px"
m

In [ ]:
from coastpy.geo.utils import get_region_of_interest_from_map

roi = get_region_of_interest_from_map(m, default_extent=(4.757, 53.108, 5.042, 53.272))
west, south, east, north = list(roi.total_bounds)

## Find the tiles for your region of interest

In [ ]:
items = gpd.sjoin(snapshot, roi).drop(columns=["index_right"])
items = list(stac_geoparquet.to_item_collection(items))

## Read data

In [ ]:
ds = odc.stac.load(items, bands=["nir"], patch_url=lambda url: f"{url}?{sas_token}")

In [ ]:
nir = ds.median("time")["nir"]

In [ ]:
nir = nir.where(lambda xx: xx != -9999)

In [ ]:
pn.Column(nir.rio.reproject(4326, nodata=np.nan).hvplot(
    x="x", y="y", geo=True, rasterize=True, tiles="ESRI"
)).show()